# ETL of legal codes.


In [6]:
from classifier import *
import os
from support import *

In [7]:
codes_folder = "../ReadFiles/Codigos"
codes_path = os.path.abspath(codes_folder)
entries = os.listdir(codes_path)

codes = []

for entry in entries:

    entry_path = os.path.join(codes_folder, entry)
    with open(entry_path, 'r', encoding='utf-8') as f:
        text = f.readlines()
        
        f.close()
    
    for line in text:
        if line == '\n':
            text.remove(line)
        if line == ' \n':
            text.remove(line)
            
    codes.append(text)

removal = codes[1][2]
removal

for code in codes:
    for line in code:
        if line == removal:
            code.remove(line)
            
for code, entry in zip(codes,entries):
    print(entry, '|', code[0])
    print(code[1:3])
    print("\n-------------------------------\n")

col_codigo_civil.txt | CÓDIGO CIVIL

['TITULO \n', 'PRELIMINAR\n']

-------------------------------

col_codigo_contencioso_administrativo.txt | Código Contencioso Administrativo

['PARTE PRIMERA\n', 'LIBRO PRIMERO\n']

-------------------------------

col_codigo_del_menor.txt | Código del Menor

['TÍTULO PRELIMINAR\n', 'Principios generales\n']

-------------------------------

col_codigo_de_comercio.txt | Código de Comercio

['TÍTULO PRELIMINAR\n', 'DISPOSICIONES GENERALES\n']

-------------------------------

col_codigo_de_construcciones_sismo_resistentes.txt | Código de Construcciones Sismo Resistentes

['TÍTULO I\n', 'OBJETO Y ALCANCE\n']

-------------------------------

col_codigo_de_la_infancia_y_la_adolescencia.txt | Código de la Infancia y la Adolescencia

['LIBRO I\n', 'LA PROTECCION INTEGRAL\n']

-------------------------------

col_codigo_de_minas.txt | Código de Minas

['TÍTULO PRIMERO\n', 'DISPOSICIONES GENERALES\n']

-------------------------------

col_codigo_de_proced

In [8]:
hierarchy = {
    # Agregados
    'LIBRO': 'book',
    'PARTE': 'part',
    
    # Original
    'TITULO' : 'headline',
    'DISPOSICIONES' : 'headline',
    'CAPITULO' : 'chapter',
    
    # Agregado
    'SECCION': , 'section',
    
    #Original
    'ARTICULO' : 'article',
    
    # Agregados
    'CONSIDERANDO' : 'article',
    'PREAMBULO' : 'article',
}
            


In [9]:
for entry, code in zip(entries, codes):
    print(entry, code[0])
    code_hierarchy = {}
    reference = 'article'
    check = 0
    while reference == 'article' and len(code)-1 > check:
        check += 1
        line = code[check]
        hint = first_word(line).upper()
        
        hint = standarize(hint)
        print(hint)
        if hint in hierarchy:
            reference = hierarchy[hint]
            
    print(f'principal_level = {reference}\n')
    print("\n-------------------------------\n")

col_codigo_civil.txt CÓDIGO CIVIL

TITULO
principal_level = headline


-------------------------------

col_codigo_contencioso_administrativo.txt Código Contencioso Administrativo

PARTE
principal_level = part


-------------------------------

col_codigo_del_menor.txt Código del Menor

TITULO
principal_level = headline


-------------------------------

col_codigo_de_comercio.txt Código de Comercio

TITULO
principal_level = headline


-------------------------------

col_codigo_de_construcciones_sismo_resistentes.txt Código de Construcciones Sismo Resistentes

TITULO
principal_level = headline


-------------------------------

col_codigo_de_la_infancia_y_la_adolescencia.txt Código de la Infancia y la Adolescencia

LIBRO
principal_level = book


-------------------------------

col_codigo_de_minas.txt Código de Minas

TITULO
principal_level = headline


-------------------------------

col_codigo_de_procedimiento_administrativo_y_de_lo_contencioso_administrativo.txt Código de Procedim